In [1]:
from app.connector import connect
from app.connector import connect_users

In [15]:
# read posts
f = open("../res/posts.csv", "r")
posts = []
for x in f:
  posts.append(x.split(","))
del posts[0]
# read posts
f = open("../res/scores.csv", "r")
scores = []
for x in f:
  scores.append(x.split(","))
del scores[0]

In [17]:
# insert all posts with user_id = dummy existing user
conn = connect()
cur = conn.cursor()

for x, y in zip(posts,scores):
    cur.execute("INSERT INTO posts(user_id, content, created_on) values(%s, %s, %s)", [66,x[1],x[0]])
    cur.execute("insert into metrics values ((SELECT post_id FROM posts ORDER BY post_id DESC LIMIT 1) , %s, %s, %s, %s, %s, %s, %s, %s, %s)", y[1:])

conn.commit()
cur.close()

Connecting to the PostgreSQL database...
PostgreSQL database version:
('PostgreSQL 12.10, compiled by Visual C++ build 1914, 64-bit',)


In [18]:
# Get all existing user_ids and post_ids
conn = connect()
cur = conn.cursor()

cur.execute("select user_id from users;")
data = cur.fetchall()
user_ids = []
for x in data:
    user_ids.append(x[0])
print("users")
print(user_ids[:5])

cur.execute("select post_id from posts;");
data = cur.fetchall()
post_ids = []
for x in data:
    post_ids.append(x[0])
print("posts")
print(post_ids[:5])

cur.close()

Connecting to the PostgreSQL database...
PostgreSQL database version:
('PostgreSQL 12.10, compiled by Visual C++ build 1914, 64-bit',)
users
[66, 88, 86, 68, 78]
posts
[9667, 9668, 9669, 9670, 9671]


In [19]:
# Assign random user_id for every post
import random
conn = connect()
cur = conn.cursor()

for post_id in post_ids:
    user_id = random.choice(user_ids[:100])
    cur.execute("update posts set user_id = %s where post_id = %s;", [user_id, post_id])

conn.commit()
cur.close()

Connecting to the PostgreSQL database...
PostgreSQL database version:
('PostgreSQL 12.10, compiled by Visual C++ build 1914, 64-bit',)


In [20]:
# populate reports table
conn = connect()
cur = conn.cursor()
reasons = ["PROFANITY", "RACE", "RELIGION", "SEX", "OTHER", "NONE"]

# Create a total of 50 000 reports
for x in range(50000):
    post_id = random.choice(post_ids[:1500])
    cur.execute("select user_id from posts where post_id = %s;", [post_id])
    poster_id = cur.fetchone()[0]
    reason = random.choice(reasons)
    cur.execute("insert into reports (poster_id, post_id, reason) values (%s, %s, %s);",
     [poster_id, post_id, reason])

conn.commit()
cur.close()

Connecting to the PostgreSQL database...
PostgreSQL database version:
('PostgreSQL 12.10, compiled by Visual C++ build 1914, 64-bit',)


<h1> Update users profile pictures </h1>
Algorithm:

1. Get all user_ids from users.users

2. Assign a picture for every user_id

3. Store assignment

4. Use the same assignments to update posts.users

In [26]:
# Get list of picture links
f = open("../res/photos.csv", "r")
pictures = []
for x in f:
  pictures.append(x)
del pictures[0]

In [27]:
# Get list of users from users.users
conn = connect_users()
cur = conn.cursor()

cur.execute("select user_id from users;")
data = cur.fetchall()
user_ids = []
for x in data:
    user_ids.append(x[0])

cur.close()

Connecting to the PostgreSQL database...
PostgreSQL database version:
('PostgreSQL 12.10, compiled by Visual C++ build 1914, 64-bit',)


In [28]:
# Assign photo for every user_id
pictures_dict = {}
for x,y in zip(user_ids, pictures):
    pictures_dict[x] = y

In [31]:
# Update users.users database
conn = connect_users()
cur = conn.cursor()

for x in pictures_dict:
    cur.execute("update users set google_photo = %s where user_id = %s", [pictures_dict[x], x])

conn.commit()
cur.close()

Connecting to the PostgreSQL database...
PostgreSQL database version:
('PostgreSQL 12.10, compiled by Visual C++ build 1914, 64-bit',)


In [33]:
# Update posts.users database
conn = connect()
cur = conn.cursor()

for x in pictures_dict:
    cur.execute("update users set google_photo = %s where user_id = %s", [pictures_dict[x], x])

conn.commit()
cur.close()

Connecting to the PostgreSQL database...
PostgreSQL database version:
('PostgreSQL 12.10, compiled by Visual C++ build 1914, 64-bit',)


In [5]:
# Update user profile pictures
conn = connect_users()
cur = conn.cursor()

cur.execute("select * from users;")
print(cur.fetchone())

conn.commit()
cur.close()

Connecting to the PostgreSQL database...
PostgreSQL database version:
('PostgreSQL 12.10, compiled by Visual C++ build 1914, 64-bit',)
(1232, 'Shayna_Bednar45', 'Hortense16@hotmail.com', 'Shayna_Bednar45, Hortense16@hotmail.com', 'https://cdn.fakercloud.com/avatars/arthurholcombe1_128.jpg', datetime.datetime(2022, 5, 1, 11, 44, 5, 627756), True, False)


<h1> Testing Area </h1>

In [14]:
categ_dict = {
    "Hate": "hate_score",
    "Normal": "normal_score",
    "Offensive": "offensive_score",
    "Profanity": "profanity_score",
    "Race": "race_score",
    "Religion": "religion_score",
    "Sex": "sex_score",
    "Other": "other_score",
    "None": "none_score",
}

filters = [{"category":"Hate", "min":0, "max":50},{"category":"Normal", "min":0, "max":100}]

def build_query(filters, name):
    query = "SELECT user_id, username from users where username like '%" + name + "%'"
    # Add filters
    if len(filters) == 0:
        return query + ";"
    else:
        for x in filters:
            categ = categ_dict[x["category"]]
            floor = str(x["min"]/100)
            ceil = str(x["max"]/100)
            query += " and (select avg("+ categ +") from metrics inner join posts using(post_id) where " \
                "posts.user_id = users.user_id) between " + floor + " and " + ceil
    return query + ";"
    
build_query(filters, "")

"SELECT user_id, username from users where username like '%%' and (select avg(hate_score) from metrics inner join posts using(post_id) where posts.user_id = users.user_id) between 0.0 and 0.5 and (select avg(normal_score) from metrics inner join posts using(post_id) where posts.user_id = users.user_id) between 0.0 and 1.0;"

SELECT user_id, username from users where username like '%%' 

and (select avg(hate_score) from metrics inner join posts using(post_id) where posts.user_id = users.user_id ) between .29 and .30;